In [1]:
from docplex.mp.model import Model
import docplex.cp.utils_visu as visu
from cplex.exceptions import CplexError 

import pandas as pd
import numpy as np

In [2]:
#number of intersection
n = 8

#exponential power
p = 1

#speed (mph) 
v= 35 

#cycle length
C= 120

#cycle per second
z = 1/C


In [ ]:
# #network details read from excel file
# data = pd.read_excel("C:\\Users\\ashams\\Box\\research works\\comparison signal offset optimization\\bandwidth maximization\\multiband.xlsx", sheet_name = "Sheet6")

# data.columns = ['Intersection ', 'vol', 'vol_bar', 's', 's_bar', 'k', 'd', 'd_bar', 'L', 'L_bar', 'R', 'tau', 'tau_bar', 'delta']

In [ ]:
# data.loc[:, 'r'] = data['R']+data['L_bar']
# data.loc[:, 'r_bar'] = data['R']+data['L']

# data.loc[:, 's'] = data.loc[:, 's']*(C-data.loc[:, 'r'])/C
# data.loc[:, 's_bar'] = data.loc[:, 's_bar']*(data.loc[:, 'r_bar'])/C

# data.loc[:, 't'] = data['d']/(v*1.47)
# data.loc[:, 't_bar'] = data['d_bar']/(v*1.47)

# data.loc[:, 'a'] = np.power(data['vol']/data['s'],p)
# data.loc[:, 'a_bar'] = np.power(data['vol_bar']/data['s_bar'],p)

# # data.loc[:, 'k'] = data.loc[:, 'vol']/ data.loc[:, 'vol_bar']

In [ ]:
# data

In [5]:
mdl1 = Model(name = 'MAXBAND')

In [6]:
#variables
b     = mdl1.continuous_var(name ='b')
m     = mdl1.integer_var_dict([i for i in range(n-1)], lb = None, name = 'm')
w     = mdl1.continuous_var_dict([i for i in range(n)], name = 'w')
w_bar = mdl1.continuous_var_dict([i for i in range(n)], name = 'w_bar')

In [7]:
#objective function
mdl1.maximize(b)

In [8]:
# #constraints
mdl1.add_constraints(w[i]+b <= C- data2['r'][i] for i in range(n))
mdl1.add_constraints(w_bar[i]+b <= C- data2['r_bar'][i] for i in range(n))

for i in range(n-1):
    mdl1.add_constraint(w[i]+w_bar[i] - w[i+1] - w_bar[i+1] + data2['t'][i] + data2['t_bar'][i] + data2['delta_diff'][i] == \
                        -0.5*(data2['r'][i] + data2['r_bar'][i]) + 0.5*(data2['r'][i+1] + data2['r_bar'][i+1]) +m[i]*C)



In [9]:
mdl1.print_information()

Model: MAXBAND
 - number of variables: 24
   - binary=0, integer=7, continuous=17
 - number of constraints: 23
   - linear=23
 - parameters: defaults
 - objective: maximize
 - problem type is: MILP


In [10]:
mdl1.solve(log_output=False)
mdl1.print_solution()

objective: 28.800
  b=28.800
  m_1=1
  m_5=1
  w_0=33.800
  w_1=46.200
  w_4=30.150
  w_5=47.950
  w_6=2.750
  w_7=16.500
  w_bar_0=33.800
  w_bar_1=37.200
  w_bar_3=4.100
  w_bar_4=30.150
  w_bar_5=47.950
  w_bar_6=2.750
  w_bar_7=16.500


In [11]:
result_w     = np.array(list(mdl1.solution.get_value_dict(w_bar).values()))
red = np.array(data2["r_bar"])

rel_offsets = 1/2 * (red[:-1] - red[1:]) + result_w[:-1] - result_w[1:] + data2["t"][:-1]
offsets = np.hstack([0, np.cumsum(rel_offsets)])%C

print(offsets)
np.round(offsets, 0)

[  0.     5.    59.5   57.7   55.75  55.75 115.75 115.5 ]


array([  0.,   5.,  60.,  58.,  56.,  56., 116., 116.])

In [ ]:
# mdl2 = Model(name='Maxband2')

In [ ]:
# #variables
# b         = mdl2.continuous_var()
# b_bar     = mdl2.continuous_var()
# m         = mdl2.integer_var_dict([i for i in range(n-1)], lb = None)
# w         = mdl2.continuous_var_dict([i for i in range(n)])
# w_bar     = mdl2.continuous_var_dict([i for i in range(n)])
# Delta     = mdl2.binary_var_dict([i for i in range(n)])
# Delta_bar = mdl2.binary_var_dict([i for i in range(n)])

In [ ]:
# k = 2
# mdl2.maximize(b + k*b_bar)

In [ ]:
# mdl2.add_constraint((1-k)*b_bar>=(1-k)*k*b)
# mdl2.add_constraints(w[i]+b <= C- data['r'][i] for i in range(n))
# mdl2.add_constraints(w_bar[i]+b <= C- data['r_bar'][i] for i in range(n))

# for i in range(n-1):
#     mdl2.add_constraint(w[i]+w_bar[i] - (w[i+1] + w_bar[i+1]) + data['t'][i] + data['t_bar'][i] + \
#                         Delta[i]*data['L'][i] - Delta_bar[i]*data['L_bar'][i]  -\
#                         (Delta[i+1]*data['L'][i+1] - Delta_bar[i+1]*data['L_bar'][i+1]) - m[i] == \
#                         (data['r'][i+1] - data['r'][i]) + data['tau_bar'][i] + data['tau'][i+1])



In [ ]:
# mdl2.print_information()

In [ ]:
# mdl2.solve()
# mdl2.print_solution()

In [ ]:
# mdl3 = Model(name='Multiband')

In [ ]:
# #variables
# b         = mdl3.continuous_var_dict([i for i in range(n)])
# b_bar     = mdl3.continuous_var_dict([i for i in range(n)])
# m         = mdl3.integer_var_dict([i for i in range(n-1)], lb = None)
# w         = mdl3.continuous_var_dict([i for i in range(n)])
# w_bar     = mdl3.continuous_var_dict([i for i in range(n)])
# Delta     = mdl3.binary_var_dict([i for i in range(n)])
# Delta_bar = mdl3.binary_var_dict([i for i in range(n)])

In [ ]:
# mdl3.maximize(mdl3.sum(data['a'][i]*b[i] + data['a_bar'][i]*b_bar[i] for i in range(n)))

In [ ]:
# mdl3.print_information()

In [ ]:
# mdl3.add_constraints( (1-data['k'][i])*b_bar[i]>=(1-data['k'][i])*data['k'][i]*b[i]  for i in range(n))
# mdl3.add_constraints( b[i]/2     <= w[i]                            for i in range(n))
# mdl3.add_constraints( b_bar[i]/2 <= w_bar[i]                        for i in range(n))
# mdl3.add_constraints( w[i]       <= (C-data['r'][i])-b[i]/2         for i in range(n))
# mdl3.add_constraints( w_bar[i]   <= (C-data['r_bar'][i])-b_bar[i]/2 for i in range(n))
# mdl3.add_constraints( b[i]/2     <= w[i+1]                          for i in range(n-1))
# mdl3.add_constraints( b_bar[i]/2 <= w_bar[i+1]                      for i in range(n-1))
# mdl3.add_constraints( w[i+1]     <=(C-data['r'][i+1])-b[i]/2        for i in range(n-1))
# mdl3.add_constraints( w_bar[i+1] <=(C-data['r_bar'][i+1])-b[i]/2    for i in range(n-1))


# for i in range(n-1):
#     mdl3.add_constraint(w[i]+w_bar[i] - (w[i+1] + w_bar[i+1]) + data['t'][i] + data['t_bar'][i] + \
#                         Delta[i]*data['L'][i] - Delta_bar[i]*data['L_bar'][i]  -\
#                         (Delta[i+1]*data['L'][i+1] - Delta_bar[i+1]*data['L_bar'][i+1]) - m[i] == \
#                         (data['r'][i+1] - data['r'][i]) + data['tau_bar'][i] + data['tau'][i+1])


In [ ]:
# mdl3.solve(log_output= False)
# mdl3.print_solution()

In [12]:
#network details read from excel file
data2 = pd.read_excel("C:\\Users\\ashams\\Box\\research works\\comparison signal offset optimization\\bandwidth maximization\\multiband.xlsx", sheet_name = "Sheet1")

data2.columns = ['Intersection', 'vol', 'vol_bar', 's', 's_bar', 't', 't_bar', 'r', 'r_bar', 'tau', 'tau_bar', 'delta', 'delta_diff']

In [13]:
data2.loc[:, 'a'] = np.power(data2['vol']/data2['s'],p)
data2.loc[:, 'a_bar'] = np.power(data2['vol_bar']/data2['s_bar'],p)

data2['k'] = data2['vol']/ data2['vol_bar']
data2['t'] = data2['t']%120

data2

,Intersection,vol,vol_bar,s,s_bar,t,t_bar,r,r_bar,tau,tau_bar,delta,delta_diff,a,a_bar,k
0,1,1175,1294,5550,5550,12.9,12.9,45,45,5,5,0.0,-5.5,0.211712,0.233153,0.908037
1,2,1303,1303,5550,5550,12.8,12.8,45,54,5,5,5.5,5.0,0.234775,0.234775,1.000000
2,3,1241,1273,5550,5550,12.8,12.8,42,45,5,5,1.0,5.0,0.223604,0.229369,0.974863
3,4,1437,1236,5550,5550,13.1,13.1,74,66,5,5,4.0,4.0,0.258919,0.222703,1.162621
4,5,1638,1458,5550,5550,12.8,12.8,44,44,5,5,0.0,0.0,0.295135,0.262703,1.123457
5,6,1694,1434,5550,5550,12.8,12.8,34,34,5,5,0.0,0.0,0.305225,0.258378,1.181311
6,7,1729,1441,5550,5550,25.5,25.5,30,30,5,5,0.0,0.5,0.311532,0.259640,1.199861
7,8,1593,1458,5550,5550,0.0,0.0,54,54,5,5,0.5,0.0,0.287027,0.262703,1.092593


In [14]:
mdl4 = Model(name='Multiband2') ### fixed cycle length, travel time and phase sequence

In [15]:
#variables
b     = mdl4.continuous_var_dict([i for i in range(n-1)], name = "b")
b_bar = mdl4.continuous_var_dict([i for i in range(n-1)], name = "b_bar")
m     = mdl4.integer_var_dict([i for i in range(n-1)], lb = None, name = "m")
w     = mdl4.continuous_var_dict([i for i in range(n)], name =  "w")
w_bar = mdl4.continuous_var_dict([i for i in range(n)], name = "w_bar")

In [16]:
mdl4.maximize(mdl4.sum(data2['a'][i]*b[i] + data2['a_bar'][i]*b_bar[i] for i in range(n-1)))

In [17]:
mdl4.print_information()

Model: Multiband2
 - number of variables: 37
   - binary=0, integer=7, continuous=30
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: maximize
 - problem type is: MILP


<h1>Multi-Band Formulation</h1>
    
    
Find- $b_i$, $\overline{b_i}$, $w_i$, $\overline{w_i}$, $m_i$-

max $\zeta = max B = \sum_{i=1}^{n-1}(a_ib_i+\overline{a_i}\overline{b_i})$

Here, $a_i = (\frac{V_i}{S_i})^p$

$V_i = $ directional volume

$S_i = $ Saturation flow

$ p = \{0,1,2,4\}$

Subject to-

$(1-k_i)\overline{b_i}\geq (1-k_i)k_ib_i$

$(1/2)b_i\leq w_i \leq (C-r_i)-(1/2)b_i$ for $i = 1, \dots, n-1$

$(1/2)b_i \leq w_{i+1} \leq (C-r_{i+1})-(1/2)b_i$  for $i = 1, \dots, n-1$

$(1/2)\overline{b}_i\leq \overline{w}_i \leq (C-\overline{r}_i)-(1/2)\overline{b}_i$ for $i = 1, \dots, n-1$

$(1/2)\overline{b}_i \leq \overline{w}_{i+1} \leq (C-\overline{r}_{i+1})-(1/2)\overline{b}_i)$  for $i = 1, \dots, n-1$

$(w_i + \overline{w}_i) - (w_{i+1}+\overline{w}_{i+1}) + (t_i + \overline{t}_i) + \Delta_i - \Delta_{i+1} - m_iC = -0.5(r_i+\overline{r}_i) + 0.5(r_{i+1}+\overline{r}_{i+1})  + (\overline{\tau}_i+\tau_i)$ for $i = 1, \dots, n-1$
    
$b_i, \overline{b}_i, w_i, \overline{w}_i \geq 0$

$m_i$ integer


In [18]:
mdl4.add_constraints( (1-data2['k'][i])*b_bar[i]>=(1-data2['k'][i])*data2['k'][i]*b[i]  for i in range(n-1))
mdl4.add_constraints( b[i]/2     <= w[i]                              for i in range(n-1))
mdl4.add_constraints( b_bar[i]/2 <= w_bar[i]                          for i in range(n-1))
mdl4.add_constraints( w[i]       <= (C-data2['r'][i])-b[i]/2          for i in range(n-1))
mdl4.add_constraints( w_bar[i]   <= (C-data2['r_bar'][i])-b_bar[i]/2  for i in range(n-1))
mdl4.add_constraints( b[i]/2     <= w[i+1]                            for i in range(n-1))
mdl4.add_constraints( b_bar[i]/2 <= w_bar[i+1]                        for i in range(n-1))
mdl4.add_constraints( w[i+1]     <= (C-data2['r'][i+1])-b[i]/2        for i in range(n-1))
mdl4.add_constraints( w_bar[i+1] <= (C-data2['r_bar'][i+1])-b[i]/2    for i in range(n-1))


for i in range(n-1):
    mdl4.add_constraint(w[i]+w_bar[i] - (w[i+1] + w_bar[i+1]) + data2['t'][i] + data2['t_bar'][i] + \
                        data2['delta_diff'][i] - m[i]*C == \
                        -0.5*(data2['r'][i] + data2['r_bar'][i]) + 0.5*(data2['r'][i+1] + data2['r_bar'][i+1]))


In [19]:
mdl4.solve(log_output= False)
mdl4.print_solution()

objective: 148.386
  b_0=38.316
  b_1=23.058
  b_2=14.221
  b_3=14.221
  b_4=66.158
  b_5=44.558
  b_6=44.558
  b_bar_0=75.000
  b_bar_1=57.000
  b_bar_2=13.863
  b_bar_3=13.863
  b_bar_4=74.326
  b_bar_5=52.637
  b_bar_6=41.326
  m_2=1
  m_6=1
  w_0=40.042
  w_1=55.842
  w_2=66.471
  w_3=7.110
  w_4=33.079
  w_5=52.921
  w_6=67.721
  w_7=22.279
  w_bar_0=37.500
  w_bar_1=37.500
  w_bar_2=63.471
  w_bar_3=6.932
  w_bar_4=37.163
  w_bar_5=52.921
  w_bar_6=67.721
  w_bar_7=20.663


In [20]:
result_w     = np.array(list(mdl4.solution.get_value_dict(w).values()))
red = np.array(data2["r"])

rel_offsets = 1/2 * (red[:-1] - red[1:]) + result_w[:-1] - result_w[1:] + data2["t"][:-1]
offsets = np.hstack([0, np.cumsum(rel_offsets)])%C

print(offsets)
np.round(offsets, 0)

[  0.         117.1          0.77095808  56.93159028  59.06287425
  57.02095808  57.02095808 115.96287425]


array([  0., 117.,   1.,  57.,  59.,  57.,  57., 116.])

In [123]:
result_w_bar     = np.array(list(mdl4.solution.get_value_dict(w_bar).values()))
red = np.array(data2["r_bar"])

rel_offsets = 1/2 * (red[:-1] - red[1:]) + result_w_bar[:-1] - result_w_bar[1:] + data2["t_bar"][:-1]
offsets = np.hstack([0, np.cumsum(rel_offsets)])%C

print(offsets)
np.round(offsets, 0)

[  0.           4.45       119.7         58.71560938  53.1
  54.7         54.7        114.7       ]


array([  0.,   4., 120.,  59.,  53.,  55.,  55., 115.])